In [ ]:
!pip install pytelegrambotapi
!pip install ISR
!git clone https://github.com/nastyakrass/Folder
  
# бот доступен по адресу: @Enotyabot
# создаем бота
TOKEN = '1492842274:AAFIUViv4tTuRBS3hPJ_Qq4zdmS-vHxJUZ4'
import telebot;
bot = telebot.TeleBot(TOKEN);
#bot = telebot.TeleBot('1420123353:AAH4y9RRjvPUgZOvht2QayuAHXK1_V3tJ4k');

from telebot.util import async_dec

import io
import math
import torchvision.models as models
import random
import numpy as np
import os
import copy
import torch
import torchvision.transforms as transforms
import PIL
from PIL import Image
import math
import random
import numpy as np
import os
import copy
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
'''
Создадим модели, необходимые для обработки картинок
'''

# загружаем GAN для суперрезолюции
from ISR.models import RRDN

gan = RRDN(weights='gans')

imsize = 256

# напишем загрузчик
def image_loader(image_name, imsize):
    # приводим картинку к нужному нам размеру, переводим в тензор
    loader = transforms.Compose([
    transforms.Resize(imsize),  
    transforms.CenterCrop(imsize),
    transforms.ToTensor()])
    # загружаем
    image = Image.open(image_name)
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# "Универсальная масочная функция". Работает быстрее написанных вручную масок, т.к. тензор маски не создается
# каждый раз с нуля
host = 'Folder/Masks/'
def masky(input, mask_type):
    _, f, h, _ = input.size()
    # загружаем файл с маской
    maskf = image_loader(host+mask_type+".bmp", h).cpu()
    mask = torch.tensor(np.array(maskf > 0.5, np.float32))
    # стакаем маску по числу фича-мапов у инпута          
    mask = mask.repeat(1, f, 1, 1)
    return mask.to(device)
   

# создаем зеркальную ей маску для применения второго стиля
def masky_dual(input, mask_type):
    _, f, h, _ = input.size()
    maskf = image_loader(host+mask_type+".bmp", h).cpu()
    mask = torch.tensor(np.array(maskf < 0.5, np.float32))          
    mask = mask.repeat(1, f, 1, 1)
    return mask.to(device)

# класс контент-лосса
class ContentLoss(nn.Module):

        def __init__(self, target,):
            super(ContentLoss, self).__init__()
            # это константа. Убираем ее из дерева вычислений
            self.target = target.detach()
            # инициализируем лосс
            self.loss = F.mse_loss(self.target, self.target )

        def forward(self, input):
            self.loss = F.mse_loss(input, self.target)
            return input

def gram_matrix(input):
        # Второй параметр -- число feature maps
        # (h,w) -- это размеры фича мапы (N=h*w)
        batch_size, f_map_num, h, w = input.size()  # batch size(=1)
        features = input.view(batch_size*f_map_num, h*w)  # преобразуем выход в вектор
        G = torch.mm(features, features.t())  # вычисляем ненормированную матрицу Грама

        # нормализуем элементы матрицы Грама, поделив на число элементов в каждой фича мапе
        return G.div(batch_size * h * w * f_map_num)

# класс для стилевых лоссов
class StyleLoss(nn.Module):
        def __init__(self, target_feature, style, option, mask_type = None):
            super(StyleLoss, self).__init__( )
            self.target = gram_matrix(target_feature).detach()
            self.style = style
            self.option = option
            self.mask_type = mask_type
            # инициализируем лосс
            self.loss = F.mse_loss(self.target, self.target)

        def forward(self, input):
            # прописываем применение выбранной маски и подсчет лосса 
            # для переноса разных стилей на разные части картинки 
            # (т.н. partial_style_transfer)
            if self.option == 'partial_style':
                if self.style == 1:
                    mask = masky(input, self.mask_type)
                elif self.style == 2:
                    mask = masky_dual(input, self.mask_type)     
                else:
                    raise RuntimeError('Incorrect style initialiation: {}'.format(self.style))     
                input1 = input.clone()
                input2 = input1 * mask
                G = gram_matrix(input2)
                self.loss = F.mse_loss(G, self.target)
            # прописываем подсчет лосса для одновременного переноса двух стилей 
            # на всю картинку 
            # (т.н. dual style transfer)
            elif self.option == 'dual_style':
                G = gram_matrix(input)
                self.loss = F.mse_loss(G, self.target) 
            else:
                raise RuntimeError('Unknown option for style: {}'.format(self.option))  
            return input

# вычисляем среднее и стандартное отклонение
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

# класс нормализации
class Normalization(nn.Module):
        def __init__(self, mean, std):
            super(Normalization, self).__init__()
            # .view среднее и стандартное отклонение к виду  [C x 1 x 1], чтобы
            # мы смогли вычислить их для стандартного тензора вида [B x C x H x W].
           
            self.mean = torch.tensor(mean).view(-1, 1, 1)
            self.std = torch.tensor(std).view(-1, 1, 1)

        def forward(self, img):
            # нормализуем картинку
            return (img - self.mean) / self.std

# загрузим предобученную vgg-модель
cnn = models.vgg19(pretrained=True).features[0:8].to(device).eval()

# создадим класс самой модели,осуществляющей перенос стиля в разных режимах: 
# один стиль (частный случай),  2 стиля одновременно, 2 стиля с маской
class MyStyleModel(nn.Module):
    def __init__(self, input_img, cnn, normalization_mean, normalization_std,
                      style1_img, style2_img, content_img, option, mask_type):
        super(MyStyleModel, self).__init__()
        self.input_img = input_img
        self.cnn = cnn
        self.normalization_mean = normalization_mean
        self.normalization_std = normalization_std
        self.style1_img = style1_img
        self.style2_img = style2_img
        self.content_img = content_img
        self.option = option
        self.mask_type = mask_type
        #self.style = 0

    # задаем функцию, возращающую модель переноса стиля и стилевые и контент-лоссы
    def get_style_model_and_losses(self):
        content_layers= ['conv_4']
        style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
        global cnn
        cnn = copy.deepcopy(self.cnn)
        global style
        global model
        #global content_loss_
        content_losses = []
        style_losses1 = []
        style_losses2 = []
        # добавим в модель модуль нормализации
        normalization = Normalization(self.normalization_mean, self.normalization_std).to(device)
        model = nn.Sequential(normalization)
        i = 0  # переменная для подсчета сверточных слоев
        for layer in cnn.children():
            if isinstance(layer, nn.Conv2d):
                i += 1
                name = 'conv_{}'.format(i)
            elif isinstance(layer, nn.ReLU):
                name = 'relu_{}'.format(i)
                # Переопределим Relu уровень 
                # для корректной работы со стилевыми слоями
                layer = nn.ReLU(inplace=False)
            elif isinstance(layer, nn.MaxPool2d):
                name = 'pool_{}'.format(i)
            elif isinstance(layer, nn.BatchNorm2d):
                name = 'bn_{}'.format(i)
            else:
                raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

            model.add_module(name, layer)

            if name in content_layers:
                # добавляем в модель слой  content loss-а:
                target = model(self.content_img).detach()
                сontent_loss = ContentLoss(target)
                model.add_module("content_loss_{}".format(i), сontent_loss)
                content_losses.append(сontent_loss)
            # добавляем в модель слои style loss-ов:
            if name in style_layers:
                # если мы осуществляем перенос разных стилей на разные части картинки,
                # то введем переменную style, которая будет определять, с какой именно
                # частью картинки мы работаем -- первый стилевой слой используется 
                # для одной части изображения, а второй стилевой слой -- для другой.
                # Соответствующие части картинки выделяются маской
                # (так что здесь мы еще указываем и тип маски)
                if self.option == 'partial_style':
                # добавили первый слой 
                    style = 1 # режим первого стиля
                    target_feature1 = model(self.style1_img).detach()
                    style_loss1 = StyleLoss(target_feature1, style, self.option, self.mask_type)
                    model.add_module("style_loss1_{}".format(i), style_loss1)
                    style_losses1.append(style_loss1)
                    # добавили второй слой
                    style = 2 # режим второго стиля
                    target_feature2 = model(self.style2_img).detach()
                    style_loss2 = StyleLoss(target_feature2, style, self.option, self.mask_type)
                    model.add_module("style_loss2_{}".format(i), style_loss2)
                    style_losses2.append(style_loss2)
                    # если же мы осуществляем одновременный перенос двух стилей на всю картинку,
                    # просто создаем 2 стилевых слоя и последовательно  прогоняем  
                    # наше изображение целиком через оба слоя.
                    # маска нам здесь больше не нужна, и мы ее даже не инициализируем.
                    # Переменная style тоже не нужна, и мы ее зануляем
                elif self.option == 'dual_style':
                    style = 0 # одновременный стилевой режим
                    # добавили первый слой
                    target_feature1 = model(self.style1_img).detach()
                    style_loss1 = StyleLoss(target_feature1, style, self.option)
                    model.add_module("style_loss1_{}".format(i), style_loss1)
                    style_losses1.append(style_loss1)
                    # добавили второй слой
                    target_feature2 = model(self.style2_img).detach()
                    style_loss2 = StyleLoss(target_feature2, style, self.option)
                    model.add_module("style_loss2_{}".format(i), style_loss2)
                    style_losses2.append(style_loss2)
                else:
                    raise RuntimeError('Unrecognized style: {}'.format(self.option))

        #выбрасываем все уровни VGG-ки после последенего style loss или content loss
        for i in range(len(model) - 1, -1, -1):
            if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
                break

        model = model[:(i + 1)]

        return model, style_losses1, style_losses2, content_losses
    
    # задаем оптимизатор
    def get_input_optimizer(self):
       
        #добоваляет содержимое тензора катринки в список изменяемых оптимизатором параметров
        optimizer = optim.LBFGS([self.input_img.requires_grad_()]) 
        return optimizer

    # задаем функцию обучения модели
    def run_style_transfer(self, num_steps=750,
                        style_weight1=100000, style_weight2=100000, content_weight=1):
        print('Building the style transfer model..')
        model, style_losses1, style_losses2, content_losses = self.get_style_model_and_losses()
        optimizer = self.get_input_optimizer()
        eps = 1e-9
        print('Optimizing..')
        run = [0]
        while run[0] <= num_steps:
        # Это функция, которая вызывается во время каждого прохода, чтобы пересчитать loss.
        # Без нее ничего не получется, так как у нас своя -- не библиотечная -- функция ошибки
            def closure():
                # это для того, чтобы значения тензора картинки не выходили за пределы [0;1]
                self.input_img.data.clamp_(eps, 1-eps)
                # обнуляем градиент
                optimizer.zero_grad()

                # прогоняем картинку через модель
                model(self.input_img)

                # считаем лоссы
                style_score1 = 0
                style_score2 = 0
                content_score = 0

                for sl in style_losses1:
                    style_score1 += sl.loss
                
                for sl in style_losses2:
                    style_score2 += sl.loss

                for cl in content_losses:
                    content_score += cl.loss
                
                # взвешивание ошибки в зависимости от типа обработки картинки
                if self.option == 'partial_style':
                    style_score1 *= style_weight1
                    style_score2 *= style_weight1
                    content_score *= content_weight
                elif self.option == 'dual_style':
                    style_score1 *= style_weight1
                    style_score2 *= 0.7 * style_weight2
                    content_score *= content_weight
                else:
                    raise RuntimeError('Unrecognized style: {}'.format(self.option))
                # строим лосс
                loss = style_score1 + style_score2 + content_score
                # и вычисляем по нему градиент
                loss.backward()

                run[0] += 1
                if run[0] % 50 == 0:
                    print("run {}:".format(run))
                    print('Style Loss1 : {:4f} Style Loss2 : {:4f} Content Loss: {:4f}'.format(
                        style_score1.item(), style_score2.item(), content_score.item()))
                    print()
                

                return style_score1 + style_score2 + content_score
            # шаг градиентного спуска
            optimizer.step(closure)

        # еще раз корректируем картинку
        self.input_img.data.clamp_(eps, 1-eps)
        return self.input_img
''' 
Теперь строим асинхронного бота с бибилиотекой Телебот!
Здесь нам поможет декоратор @async_dec()
Конструкция тестировалась -- она работает асинхронно :)
'''
# функция, которая выводит на экран пользователя меню бота после каждой отработки
@async_dec()
def signal(message):
    bot.send_message(message.from_user.id, "Что теперь будем полоскать? Тазик готов!")
    keyboard = types.InlineKeyboardMarkup()
    # Делаем кнопки для различных режимов переноса стиля
    key_1st= types.InlineKeyboardButton(text='Перенесем 1 стиль', callback_data='1st')
    # И добавляем кнопку на экран
    keyboard.add(key_1st)
    key_2st= types.InlineKeyboardButton(text='Перенесем 2 стиля с маской', callback_data='2st')
    # И добавляем кнопку на экран
    keyboard.add(key_2st)
    key_doublest= types.InlineKeyboardButton(text='Перенесем одновременно 2 стиля на картинку', callback_data='double')
    # И добавляем кнопку на экран
    keyboard.add(key_doublest)
    key_gan = types.InlineKeyboardButton(text='Применим GAN', callback_data='gan')
    # И добавляем кнопку на экран
    keyboard.add(key_gan)
    # Выводим клавиатуру
    bot.send_message(message.from_user.id, text='Выбери действие, порадуй Енотика', reply_markup=keyboard)

# функция для команды "help"
@async_dec()
def signal_h(message):
    bot.send_message(message.from_user.id, "Не стесняйся -- выбирай любую понравившуюся опцию из списка и далее следуй инструкциям от Еноти. Для загрузки фоток можно пользоваться и Forward'ом. С Любовью, твой Енот"+'\U0001F496')
    keyboard = types.InlineKeyboardMarkup()
    # Клавиатура
    key_1st= types.InlineKeyboardButton(text='Перенесем 1 стиль', callback_data='1st')
    keyboard.add(key_1st)
    key_2st= types.InlineKeyboardButton(text='Перенесем 2 стиля с маской', callback_data='2st')
    keyboard.add(key_2st)
    key_doublest= types.InlineKeyboardButton(text='Перенесем одновременно 2 стиля на картинку', callback_data='double')
    keyboard.add(key_doublest)
    key_gan = types.InlineKeyboardButton(text='Применим GAN: Super Resolution', callback_data='gan')
    keyboard.add(key_gan)
    bot.send_message(message.from_user.id, text='Выбери действие, порадуй Енотика', reply_markup=keyboard)

import io
# функция выбора размера картинок (рекомендован размер 256 на 256)
@async_dec()
def size_im(message):
    bot.send_message(message.from_user.id, "Выбери для начала размер выходных картинок. Ты всегда можешь его изменить, вновь нажав на команду 'start'. С Любовью, твой Енот")
    keyboard = types.InlineKeyboardMarkup()
    key_1st= types.InlineKeyboardButton(text='256 на 256 (рекомендовано!)', callback_data='large')
    keyboard.add(key_1st)
    key_2st= types.InlineKeyboardButton(text='128 на 128', callback_data='small')
    keyboard.add(key_2st)
    bot.send_message(message.from_user.id, text='Выбери свой размерчик! (по умолчанию -- 256)', reply_markup=keyboard)

# функция перевода тензора в PIL-картинку: нужна для отправки обрабтанного изображения пользователю
def get_res(output):
    bio = io.BytesIO()
    bio.name = 'output_m.jpeg'
    unloader = transforms.ToPILImage()
    image = output.clone()   
    image = image.squeeze(0) 
    image = unloader(image)
    image.save(bio, 'jpeg')
    bio.seek(0)
    return image

# напишем функцию, которая по запросу пользователя выводит 3 ссылочки с ответами на его вопрос
from googlesearch import search
@async_dec()
def find_me(message, query):
    # результаты Google Search 
    search_result_list = list(search(query, tld="co.in", num=10, stop=3, pause=1))
    if len(search_result_list) == 0:
        bot.send_message(message.chat.id, "Енотя этого не знает :(. Попробуй спроси что-нибудь еще или переформулируй запрос" )
    else:
        bot.send_message(message.chat.id, "Держи!")
        bot.send_message(message.chat.id, search_result_list[0])
        bot.send_message(message.chat.id, search_result_list[1])
        bot.send_message(message.chat.id, search_result_list[2])

# создаем текстовые файлики для записи и считывания текущего режима обработки и т.п.
with open(host+"data.txt", "w") as f:
    f.write('BotEnot\n')
with open(host+"size.txt", "w") as f:
    f.write('256')
with open(host+"query.txt", "w") as f:
    f.write('Где живет енот')
# обработка команды "start"
@async_dec()
@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Привет, я -- Енотя, ученый Енот-полоскун, который умеет полоскать фотки разными интересными способами. К тому же, я могу делать Super Resolution картинки без потери качества. Напиши мне что-нибудь, и я порадую тебя, ведь Еноты -- они такие'+'\U0001F49B')
    # шлём приественную картинку
    with open(host+"poloskun.jpg", "rb") as file:
        data = file.read()
    bot.send_photo(message.chat.id, photo=data)
    size_im(message)

# обработка команды "help"
@async_dec()
@bot.message_handler(commands=['help'])
def start_message(message):
    bot.send_message(message.chat.id, 'Привет, ученый Енот-полоскун бежит на помощь :)')
    signal_h(message)

# обработка команды "request"
@async_dec()
@bot.message_handler(commands=['request'])
def knowledge(message):
    bot.send_message(message.chat.id, "Спроси о чем-нибудь Енотю, и он пришлет полезные ссылочки об этом!")
    with open(host+"query.txt", "w") as f:
        f.write('query')


from telebot import types
# текстовый обработчик, вывод меню
@async_dec()
@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    with open(host+'query.txt', 'r') as file:
        file_content=file.readlines()
        for line in file_content:
            ask = line.strip()
    if ask == "query":
        find_me(message, message.text)
        with open(host+"query.txt", "w") as f:
            f.write('Где живет енот')
    else:
        bot.send_message(message.from_user.id, "Сейчас я отполоскаю твою фотку так, как ты захочешь")
        bot.send_message(message.from_user.id, "Я приготовил тазик :-)")
        keyboard = types.InlineKeyboardMarkup()
        key_1st= types.InlineKeyboardButton(text='Перенесем 1 стиль', callback_data='1st')
        keyboard.add(key_1st)
        key_2st= types.InlineKeyboardButton(text='Перенесем 2 стиля с маской', callback_data='2st')
        keyboard.add(key_2st)
        key_doublest= types.InlineKeyboardButton(text='Перенесем одновременно 2 стиля на картинку', callback_data='double')
        keyboard.add(key_doublest)
        key_gan = types.InlineKeyboardButton(text='Применим GAN', callback_data='gan')
        keyboard.add(key_gan)
        bot.send_message(message.from_user.id, text='Выбери действие, порадуй Енотика', reply_markup=keyboard)

# обработка нажатий на виртуальные кнопки выбора режимов обработки, размера картинок, вида масок и т.п.
@async_dec()
@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    if call.data == "large":
        bot.send_message(call.message.chat.id, "Вызов принят!")
        bot.send_message(call.message.chat.id, "Теперь напиши мне что-нибудь хорошее, а то я не буду полоскать! Или жми на косую черту внизу строки ввода и выбирай команду 'Помощь Полоскуна'")
        with open(host+"size.txt", "w") as f:
            f.write('256')
        bot.answer_callback_query(call.id, text  = " ") 
    if call.data == "small":
        bot.send_message(call.message.chat.id, "Ух, минималист!")
        bot.send_message(call.message.chat.id, "Теперь напиши мне что-нибудь хорошее, а то я не буду полоскать! Или жми на косую черту внизу строки ввода и выбирай команду 'Помощь Полоскуна'")
        with open(host+"size.txt", "w") as f:
            f.write('128') 
        bot.answer_callback_query(call.id, text  = " ")
    if call.data == "gan":
        bot.send_message(call.message.chat.id, "Загрузи свою фотку на обработку!")
        with open(host+"data.txt", "w") as f:
            f.write('Gan\n') 
        bot.answer_callback_query(call.id, text  = " ") 
    if call.data == "double":
        bot.send_message(call.message.chat.id, "Загрузи свою фотку на обработку!")
        with open(host+"data.txt", "w") as f:
            f.write('Double\n') 
        bot.answer_callback_query(call.id, text  = " ")   
    elif call.data == "1st":
        bot.send_message(call.message.chat.id, "Загрузи свою фотку на обработку!")
        with open(host+"data.txt", "w") as f:
            f.write('1st\n')   
        bot.answer_callback_query(call.id, text  = " ")
    elif call.data == "2st":
        with open(host+"data.txt", "w") as f:
            f.write('2st\n') 
        keyboard = types.InlineKeyboardMarkup()
        # клавиатура для выбора типов масок: доступно 4 опции
        key_1st= types.InlineKeyboardButton(text='Диагональная маска', callback_data='diag')
        keyboard.add(key_1st)
        key_2st= types.InlineKeyboardButton(text='Интересная маска', callback_data='interest')
        keyboard.add(key_2st)
        key_doublest= types.InlineKeyboardButton(text='Вертикальная маска', callback_data='vert')
        keyboard.add(key_doublest)
        key_gan = types.InlineKeyboardButton(text='Горизонтальная маска', callback_data='hor')
        keyboard.add(key_gan)
        bot.send_message(call.message.chat.id, text='Выбери масочку, не серди Енотика', reply_markup=keyboard)  
        bot.answer_callback_query(call.id, text  = " ")
        
    elif call.data == "diag":
        with open(host+"mask.txt", "w") as f:
            f.write('diag\n') 
        bot.send_message(call.message.chat.id, "А вот теперь загрузи свою фотку на обработку!")
        bot.answer_callback_query(call.id, text  = " ")
    elif call.data == "interest":
        with open(host+"mask.txt", "w") as f:
            f.write('interesting\n') 
        bot.send_message(call.message.chat.id, "А вот теперь загрузи свою фотку на обработку!")
        bot.answer_callback_query(call.id, text  = " ")
    elif call.data == "hor":
        with open(host+"mask.txt", "w") as f:
            f.write('hor\n') 
        bot.send_message(call.message.chat.id, "А вот теперь загрузи свою фотку на обработку!")
        bot.answer_callback_query(call.id, text  = " ")
    elif call.data == "vert":
        with open(host+"mask.txt", "w") as f:
            f.write('vert\n') 
        bot.send_message(call.message.chat.id, "А вот теперь загрузи свою фотку на обработку!")
        bot.answer_callback_query(call.id, text  = " ")

# обработка неподдерживаемого контента
@async_dec()
@bot.message_handler(content_types=['audio', 'document'])        
def content(message):
    bot.send_message(message.from_user.id, "Енотя такое не полоскает! Следуй моим инструкциям или нажми на косую черту в правом углу строки ввода и выбери команду 'Помощь Полоскуна' ")    

 # обработка стикеров
@async_dec()
@bot.message_handler(content_types=['sticker'])
def get_reply(message):
    signal_h(message)
# переключатели режимов обработки фото
s = 0
v = 0
l = 0
m = 0
# обработчик фото
@async_dec()
@bot.message_handler(content_types=['photo'])        
def photo(message):
    global fileID
    global fileID1
    global fileID2
    global content_img
    global style1_img
    global s
    global v
    global m
    global l
    global mask
    global imsize
    # считываем текущий размер картинки
    with open(host+'size.txt', 'r') as file:
        file_content=file.readlines()
        for line in file_content:
            imsize = int(line.strip())
    # считываем текущий режим обработки
    with open(host+'data.txt', 'r') as file:
        file_content=file.readlines()
        for line in file_content:
            vals = line.strip()
    # Если выбран режим "Gan : SuperResolution"
    if vals == 'Gan':
        # загружаем фото на обработку
        if (v == 0) :
            bot.send_message(message.from_user.id, "Жду фотку на обработку!")
            print('Superresolution')
            fileID = message.photo[-1].file_id
            file_info = bot.get_file(fileID)
            downloaded_file = bot.download_file(file_info.file_path)
            img = Image.open(io.BytesIO(downloaded_file))
            v += 1
            bot.send_message(message.from_user.id, "Енотя доволен!")
            print(v)
        # обрабатываем
        if v == 1:
            bot.send_message(message.from_user.id, "Енотя полоскает...")
            v = 0
            # выводим результат обычной бикубической интерполяции -- понижает размерность
            if imsize == 128:
                img = img.resize((img.size[0]-35, img.size[1]-35), Image.BICUBIC)
            elif imsize == 256:
                img = img.resize((img.size[0]-120, img.size[1]-120), Image.BICUBIC)
            bio = io.BytesIO()
            bio.name = 'output.jpeg'
            img.save(bio, 'jpeg')
            bio.seek(0)
            bot.send_message(message.from_user.id, "Посмотри  на результат обычной бикубической интерполяции (я сжал фото для более наглядной демонстрации)")
            bot.send_photo(message.from_user.id, photo=img)
            # а теперь применяем GAN  и выводим результат
            lr_img = np.array(img, np.float64)
            sr_img = gan.predict(lr_img, by_patch_of_size=50)
            u = Image.fromarray(sr_img)
            bio = io.BytesIO()
            bio.name = 'output_f.jpeg'
            u.save(bio, 'jpeg')
            bio.seek(0)
            bot.send_message(message.from_user.id, "А теперь взгляни на работу GAN")
            bot.send_photo(message.from_user.id, photo=u)
            # радуемся :)
            bot.send_message(message.from_user.id, "Видишь, Енотя не врет, качество Super resolution действительно классное!")
            bot.send_message(message.from_user.id, "За пушистый хвост енотий поднимем бокальчик!" + "\U0001F942")
            # выводим меню
            signal(message)
    # Если выбран режим "Перенесем 2 стиля с маской"
    if vals == '2st':
        print(s)
        print('With mask')
        # загружаем фото на обработку
        if (s == 0) :
            bot.send_message(message.from_user.id, "Жду фотку на обработку!")
            with open(host+'mask.txt', 'r') as file:
                file_content=file.readlines()
                for line in file_content:
                    mask = line.strip()
            fileID = message.photo[-1].file_id
            file_info = bot.get_file(fileID)
            downloaded_file = bot.download_file(file_info.file_path)
            img = io.BytesIO(downloaded_file)
            content_img = image_loader(img, imsize)
            s += 1
            bot.send_message(message.from_user.id, "Енотя доволен!")
        # загружаем первую стилевую фотку   
        if (s == 1):
            fileID1 = message.photo[-1].file_id
            if (fileID != fileID1):
                bot.send_message(message.from_user.id, "Енотя очень доволен!")   
                file_info = bot.get_file(fileID1)
                downloaded_file = bot.download_file(file_info.file_path)
                img1 = io.BytesIO(downloaded_file)
                style1_img = image_loader(img1, imsize)
                s += 1
        if s == 1:
            bot.send_message(message.from_user.id, "Теперь загрузи первую стилевую фотку на обработку!") 
        # загружаем вторую стилевую фотку 
        if (s == 2):
            fileID2 = message.photo[-1].file_id    
            if (fileID != fileID1) and  (fileID != fileID2) and  (fileID1 != fileID2):
                bot.send_message(message.from_user.id, "Енотя чрезвычайно доволен!")   
                file_info = bot.get_file(fileID2)
                downloaded_file = bot.download_file(file_info.file_path)
                img2 = io.BytesIO(downloaded_file)
                style2_img = image_loader(img2, imsize)
                s += 1
        if 2 <= s < 3:
            bot.send_message(message.from_user.id, "Теперь загрузи вторую стилевую фотку на обработку!") 
        # обрабатываем
        if s == 3:
            bot.send_message(message.from_user.id, "Енотя полоскает...")
            s = 0
            input_img = content_img.clone()
            # создаем экземпляр нашего класса для обработки с маской и переносим стиль
            st_transf_model = MyStyleModel(input_img, cnn, cnn_normalization_mean, cnn_normalization_std,
                      style1_img, style2_img, content_img, option ='partial_style', mask_type = mask)
            output = st_transf_model.run_style_transfer(num_steps=100)
            # отправляем результат и радуемся)
            image = get_res(output)
            bot.send_photo(message.from_user.id, photo=image)
            bot.send_message(message.from_user.id, "За пушистый хвост енотий поднимем бокальчик!"+"\U0001F942")
            # выводим меню
            signal(message)
    # Если выбран режим "Перенесем 2 стиля одновременно"
    elif vals == 'Double':
        print(m)
        print('Double')
        # загружаем фото на обработку
        if (m == 0) :
            bot.send_message(message.from_user.id, "Жду фотку на обработку!")
            fileID = message.photo[-1].file_id
            file_info = bot.get_file(fileID)
            
            downloaded_file = bot.download_file(file_info.file_path)
            img = io.BytesIO(downloaded_file)
            content_img = image_loader(img, imsize)
            m += 1
            bot.send_message(message.from_user.id, "Енотя доволен!")
        # загружаем первую стилевую фотку   
        if (m == 1):
            fileID1 = message.photo[-1].file_id
            if (fileID != fileID1):
                bot.send_message(message.from_user.id, "Енотя очень доволен!")   
                file_info = bot.get_file(fileID1)
                downloaded_file = bot.download_file(file_info.file_path)
                img1 = io.BytesIO(downloaded_file)
                style1_img = image_loader(img1, imsize)
                m += 1
        
        if (m == 1):
            bot.send_message(message.from_user.id, "Теперь загрузи первую стилевую фотку на обработку!") 
        # загружаем вторую стилевую фотку 
        if (m == 2):
            fileID2 = message.photo[-1].file_id    
            if (fileID != fileID1) and  (fileID != fileID2) and  (fileID1 != fileID2):
                bot.send_message(message.from_user.id, "Енотя чрезвычайно доволен!")   
                file_info = bot.get_file(fileID2)
                downloaded_file = bot.download_file(file_info.file_path)
                img2 = io.BytesIO(downloaded_file)
                style2_img = image_loader(img2, imsize)
                m += 1
        if 2 <= m < 3:
            bot.send_message(message.from_user.id, "Теперь загрузи вторую стилевую фотку на обработку!") 
        # обрабатываем
        if m == 3:
            bot.send_message(message.from_user.id, "Енотя полоскает...")
            m = 0
            input_img = content_img.clone()
            # создаем экземпляр нашего класса для обработки с двумя стилями без маски  -- и переносим стиль
            st_transf_model = MyStyleModel(input_img, cnn, cnn_normalization_mean, cnn_normalization_std,
                      style1_img, style2_img, content_img, option='dual_style', mask_type=None)
            output = st_transf_model.run_style_transfer(num_steps=100)
            # отправляем результат и радуемся)
            image = get_res(output)
            bot.send_photo(message.from_user.id, photo=image)
            bot.send_message(message.from_user.id, "За пушистый хвост енотий поднимем бокальчик!"+"\U0001F942")
            # выводим меню
            signal(message)
    # Если выбран режим "Перенесем 1 стиль"
    elif vals == '1st':
        print(l)
        print('1 style')
        # загружаем фото на обработку
        if (l == 0) :
            bot.send_message(message.from_user.id, "Жду фотку на обработку!")
            fileID = message.photo[-1].file_id
            file_info = bot.get_file(fileID)
            downloaded_file = bot.download_file(file_info.file_path)
            img = io.BytesIO(downloaded_file)
            content_img = image_loader(img, imsize)
            l += 1
            bot.send_message(message.from_user.id, "Енотя доволен!")
        # загружаем стилевое фото
        if (l == 1):
            fileID1 = message.photo[-1].file_id
            if (fileID != fileID1):
                bot.send_message(message.from_user.id, "Енотя очень доволен!")   
                file_info = bot.get_file(fileID1)
                downloaded_file = bot.download_file(file_info.file_path)
                img1 = io.BytesIO(downloaded_file)
                style1_img = image_loader(img1, imsize)
                l += 1
        if l == 1:
            bot.send_message(message.from_user.id, "Теперь загрузи стилевую фотку на обработку!") 
        # обрабатываем
        if l == 2:
            bot.send_message(message.from_user.id, "Енотя полоскает...")
            l = 0
            input_img = content_img.clone()
            # создаем экземпляр нашего класса для обработки с одним стилевым изображением -- и переносим стиль
            st_transf_model = MyStyleModel(input_img, cnn, cnn_normalization_mean, cnn_normalization_std,
                      style1_img, style1_img, content_img, option = 'dual_style', mask_type = None)
            output = st_transf_model.run_style_transfer(num_steps=100)
            # отправляем результат и радуемся)
            img = get_res(output)
            bot.send_photo(message.from_user.id, photo=img)
            bot.send_message(message.from_user.id, "За пушистый хвост енотий поднимем бокальчик!"+"\U0001F942")
            #выводим меню
            signal(message)

# запускаем бота на лонг-поллинге
if __name__ == '__main__':
    bot.polling(none_stop=True, interval=0)



fatal: destination path 'Folder' already exists and is not an empty directory.
0
1 style
1
1 style
Building the style transfer model..
Optimizing..


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:150: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:151: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


run [50]:
Style Loss1 : 0.591317 Style Loss2 : 0.413922 Content Loss: 3.590458

run [100]:
Style Loss1 : 0.373545 Style Loss2 : 0.261482 Content Loss: 3.209065

Superresolution
1
0
1 style
1
1 style
Building the style transfer model..
Optimizing..
run [50]:
Style Loss1 : 1.514731 Style Loss2 : 1.060312 Content Loss: 10.355325

run [100]:
Style Loss1 : 0.777658 Style Loss2 : 0.544360 Content Loss: 5.774375

Superresolution
1
0
With mask
1
With mask
2
With mask
Building the style transfer model..
Optimizing..
run [50]:
Style Loss1 : 5.511429 Style Loss2 : 12.978891 Content Loss: 25.503139

run [100]:
Style Loss1 : 3.214359 Style Loss2 : 9.213955 Content Loss: 23.461500

Superresolution
1
0
Double
1
Double
2
Double
Building the style transfer model..
Optimizing..
run [50]:
Style Loss1 : 1059.885742 Style Loss2 : 1462.406738 Content Loss: 21.037395

run [100]:
Style Loss1 : 1021.306946 Style Loss2 : 1489.050659 Content Loss: 16.789764

0
Double
1
Double
2
Double
Building the style transfer